# CS 224n Assignment #3: Dependency Parsing 
In this assignment, you will build a neural dependency parser using PyTorch. For a review of the fundamentals of PyTorch, please check out the PyTorch review session on Canvas. In Part 1, you will learn about two general neural network techniques (Adam Optimization and Dropout). In Part 2, you will implement and
train a dependency parser using the techniques from Part 1, before analyzing a few erroneous dependency parses.

## 1. Machine Learning & Neural Networks (8 points) 

- (a) (4 points) Adam Optimizer  
Recall the standard Stochastic Gradient Descent update rule:  
$$𝜽 \leftarrow 𝜽 - \alpha\nabla_𝜽 J_{\text{minibatch}}(𝜽)$$  
where 𝜽 is a vector containing all of the model parameters, $J$ is the loss function, $\nabla_𝜽 J_{\text{minibatch}}(𝜽)$ is the gradient of the loss function with respect to the parameters on a minibatch of data, and $\alpha$ is the learning rate. Adam Optimization$^1$ uses a more sophisticated update rule with two additional steps.$^2$  
  - i. (2 points) First, Adam uses a trick called *momentum* by keeping track of 𝐦, a rolling average of the gradients:  
  $$\begin{align*}
  𝐦 &\leftarrow \beta_1 𝐦 + (1-\beta_1)\nabla_𝜽 J_{\text{minibatch}}(𝜽) \\
  𝜽 &\leftarrow 𝜽 - \alpha 𝐦
  \end{align*}$$  
  where $\beta_1$ 1 is a hyperparameter between 0 and 1 (often set to 0.9). Briefly explain in 2-4 sentences (you don’t need to prove mathematically, just give an intuition) how using 𝐦 stops the updates from varying as much and why this low variance may be helpful to learning, overall.  
  $\color{red}{*Answer*}$  
By using momentum, the effect of the current gradient at each step is multiplied by $(1-\beta_1)$ and the parameter updates continue to move in the direction as the previous iterations. This method can help avoid local minima. Also, the model can be robust to noisy gradients from minibatches.  
  <br/>  
  - ii. (2 points) Adam extends the idea of *momentum* with the trick of *adaptive learning rates* by keeping track of 𝐯, a rolling average of the magnitudes of the gradients:  
  $$\begin{align*}
  𝐦 &\leftarrow \beta_1 𝐦 + (1-\beta_1)\nabla_𝜽 J_{\text{minibatch}}(𝜽) \\
  𝐯 &\leftarrow \beta_2 𝐯 + (1-\beta_2)(\nabla_𝜽 J_{\text{minibatch}}(𝜽) \odot \nabla_𝜽 J_{\text{minibatch}}(𝜽)) \\
  𝜽 &\leftarrow 𝜽 - \alpha 𝐦 / \sqrt{𝐯}
  \end{align*}$$  
  where ⊙ and $/$ denote elementwise multiplication and division (so 𝐳⊙𝐳 is elementwise squaring) and $\beta_2$ is a hyperparameter between 0 and 1 (often set to 0.99). Since Adam divides the update
  by $\sqrt{𝐯}$, which of the model parameters will get larger updates? Why might this help with
learning?  
  $\color{red}{*Answer*}$  
When the gradient at the current step is large, the value of $\sqrt{𝐯}$ gets bigger and the parameters of those will be regularized to have small updates. On the other hand, if the current gradient is very small(<1), the update step will get larger. Then the model can escape a saddle point much faster.

- (b) (4 points) Dropout$^3$ is a regularization technique. During training, dropout randomly sets units in the hidden layer 𝐡 to zero with probability $p_{\text{drop}}$ (dropping different units each minibatch), and then multiplies 𝐡 by a constant $\gamma$. We can write this as:  
$$𝐡_{\text{drop}} = \gamma 𝐝 \odot 𝐡$$  
where $𝐝\in \{ 0,1\}^{D_h}$ ($D_h$ is the size of 𝐡) is a mask vector where each entry is 0 with probability $p_{\text{drop}}$ and 1 with probability (1-$p_{\text{drop}}$). $\gamma$ is chosen such that the expected value of $𝐡_{\text{drop}}$ is 𝐡:  
$$\mathbb{E}_{p_{\text{drop}}} \left[ 𝐡_{\text{drop}}\right]_i = h_i$$  
for all $i\in \{ 1,\ldots, D_h\}$.  
  - i. (2 points) What must $\gamma$ equal in terms of $p_{\text{drop}}$? Briefly justify your answer or show your math derivation using the equations given above.  
  $\color{red}{*Answer*}$  
  $$\begin{align*}\mathbb{E}_{p_{\text{drop}}} \left[ 𝐡_{\text{drop}}\right]_i &= \mathbb{E}_{p_{\text{drop}}}\left[ \gamma d_i \times h_i\right] \\
  &= \gamma \mathbb{E}_{p_{\text{drop}}}\left[ d_i \times h_i\right]\\
  &= \gamma \left[ p_{\text{drop}}\cdot 0 + (1-p_{\text{drop}}) \cdot h_i \right] \\
  &= \gamma (1-p_{\text{drop}}) h_i = h_i \\
  &\therefore \gamma = \frac{1}{(1-p_{\text{drop}})}
  \end{align*}$$  
  <br/>
  - ii. (2 points) Why should dropout be applied during training? Why should dropout __NOT__ be applied during evaluation? (Hint: it may help to look at the paper linked above in the write-up.)  
  $\color{red}{*Answer*}$  
  We use dropout to prevent overfitting in training step. If we apply dropout in test time, some neurons dropped and the learned weights of those will not contribute to evalutation.

## 2. Neural Transition-Based Dependency Parsing (44 points)  
In this section, you’ll be implementing a neural-network based dependency parser with the goal of maximizing performance on the UAS (Unlabeled Attachment Score) metric.  

Before you begin, please follow the README to install all the needed dependencies for the assignment. We will be using PyTorch 1.7.1 from <https://pytorch.org/get-started/locally/> with the `CUDA` option set to `None`, and the tqdm package – which produces progress bar visualizations throughout your training process. The official PyTorch website is a great resource that includes tutorials for understanding PyTorch’s Tensor library and neural networks.  

A dependency parser analyzes the grammatical structure of a sentence, establishing relationships between *head* words, and words which modify those heads. There are multiple types of dependency parsers, including transition-based parsers, graph-based parsers, and feature-based parsers. Your implementation will be a transition-based parser, which incrementally builds up a parse one step at a time. At every step it maintains a *partial* parse, which is represented as follows:  
- A *stack* of words that are currently being processed.
- A *buffer* of words yet to be processed.
- A list of *dependencies* predicted by the parser.  

Initially, the stack only contains ROOT, the dependencies list is empty, and the buffer contains all words of the sentence in order. At each step, the parser applies a *transition* to the partial parse until its buffer is empty and the stack size is 1. The following transitions can be applied:  
- `SHIFT`: removes the first word from the buffer and pushes it onto the stack  
- `LEFT-ARC`: marks the second (second most recently added) item on the stack as a dependent of the first item and removes the second item from the stack, adding a *first_word* → *second_word* dependency to the dependency list.
- `RIGHT-ARC`: marks the first (most recently added) item on the stack as a dependent of the second item and removes the first item from the stack, adding a *second_word* → *first_word* dependency to the dependency list.  
  
On each step, your parser will decide among the three transitions using a neural network classifier.  
- (a) (4 points) Go through the sequence of transitions needed for parsing the sentence _"I parsed this sentence correctly"_. The dependency tree for the sentence is shown below. At each step, give the configuration of the stack and buffer, as well as what transition was applied this step and what new dependency was added (if any). The first three steps are provided below as an example.  
  $\color{red}{*Answer*}$  

| Stack | Buffer | New dependency | Transition |
| :--------- | :--------- | :--------- | :--------- | 
| [ROOT] | [I, parsed, this, sentence, correctly] | | Initial Configuration |
| [ROOT, I] | [parsed, this, sentence, correctly] | | `SHIFT` |
| [ROOT, I, parsed] | [this, sentence, correctly] | | `SHIFT` |
| [ROOT, parsed] | [this, sentence, correctly] | parsed $\rightarrow$ I | `LEFT-ARC` |
| [ROOT, parsed, this] | [sentence, correctly] | | `SHIFT` |
| [ROOT, parsed, this, sentence] | [correctly] | | `SHIFT` |
| [ROOT, parsed, sentence] | [correctly] | sentence $\rightarrow$ this | `LEFT-ARC` |
| [ROOT, parsed] | [correctly] | parsed $\rightarrow$ sentence | `RIGHT-ARC` |
| [ROOT, parsed, correctly] | [] | | `SHIFT` |
| [ROOT, parsed] | [] | parsed $\rightarrow$ correctly | `RIGHT-ARC` |
| [ROOT] | [] | ROOT $\rightarrow$ parsed | `RIGHT-ARC` |
  <br/>

- (b) (2 points) A sentence containing *n* words will be parsed in how many steps (in terms of *n*)? Briefly explain in 1-2 sentences why.  

  $\color{red}{*Answer*}$  
  *2n*: In processing, each word should be added to *buffer* and removed.  
  <br/>
- (c), (d), (e)  
  __Coding__: Implement `parser_transitions.py`,  `parser_model.py`, `run.py`  

  $\color{red}{*Result*}$
  - Average Train Loss: 0.0674  
  - best dev UAS: 88.81  
  - test UAS: 89.39  
  <br/>
- (f) (12 points) For each sentence, state the type of error, the incorrect dependency, and the correct dependency. While each sentence should have a unique error type, there may be multiple possible correct dependencies for some of the sentences.  
  Here are four types of parsing error:  
  - __Prepositional Phrase Attachment Error__: a prepositional phrase is attached to the wrong head word.  
  - __Verb Phrase Attachment Error__: a verb phrase is attached to the wrong head word.  
  - __Modifier Attachment Error__: a modifier is attached to the wrong head word.  
  - __Coordination Attachment Error__: the second conjunct is attached to the wrong head word.  

  $\color{red}{*Answer*}$  
  - i. _"I disembarked and was heading to a wedding fearing my death."_  
    - __Error type__: __Verb Phrase Attachment Error__  
    - __Incorrect dependency__: *wedding* → *fearing*
    - __Correct dependency__: *heading* → *fearing*
  - ii. _"It makes me want to rush out and rescue people from dilemmas of their own making."_  
    - __Error type__: __Coordination Attachment Error__ 
    - __Incorrect dependency__: *makes* → *rescue*
    - __Correct dependency__: *rush* → *rescue*
  - iii. _"It is on loan from a guy named Joe O’Neill in Midland, Texas."_
    - __Error type__: __Prepositional Phrase Attachment Error__
    - __Incorrect dependency__: *named* → *Midland*
    - __Correct dependency__: *guy* → *Midland*
  - iv. _"Brian has been one of the most crucial elements to the success of Mozilla software."_
    - __Error type__: __Modifier Attachment Error__
    - __Incorrect dependency__: *elements* → *most*
    - __Correct dependency__: *crucial* → *most*
